In [1]:
from mpi4py import MPI
from pathlib import Path
import numpy as np
import time as tm
import math
import sys

import veloxchem as vlx

from veloxchem.veloxchemlib import (mpi_master, hartree_in_wavenumber, hartree_in_ev,
                           hartree_in_inverse_nm, fine_structure_constant,
                           extinction_coefficient_from_beta)
from veloxchem.outputstream import OutputStream
from veloxchem.distributedarray import DistributedArray
from veloxchem.linearsolver import LinearSolver
from veloxchem.sanitychecks import (molecule_sanity_check, scf_results_sanity_check,
                           dft_sanity_check, pe_sanity_check)
from veloxchem.errorhandler import assert_msg_critical
from veloxchem.checkpoint import (check_rsp_hdf5, create_hdf5,
                         write_rsp_solution_with_multiple_keys)
from veloxchem.oneeints import compute_electric_dipole_integrals

from veloxchem.veloxchemlib import ElectricDipoleMomentDriver


In [2]:
# Set up molecule and basis

water_xyz = """3 

O   0.0   0.0   0.0
H   0.0   1.4   1.1
H   0.0  -1.4   1.1
"""

molecule = vlx.Molecule.from_xyz_string(water_xyz)
basis = vlx.MolecularBasis.read(molecule, '6-31G')

scf_drv = vlx.ScfRestrictedDriver()
scf_drv.ostream.mute()
scf_drv.update_settings({"conv_thresh":1e-12})
scf_tensors = scf_drv.compute(molecule, basis)

moints_drv = vlx.MOIntegralsDriver()

In [3]:
class ComplexResponse(LinearSolver):
    """
    Implements the complex linear response solver.

    :param comm:
        The MPI communicator.
    :param ostream:
        The output stream.

    Instance variables
        - a_operator: The A operator.
        - a_components: Cartesian components of the A operator.
        - b_operator: The B operator.
        - b_components: Cartesian components of the B operator.
        - frequencies: The frequencies.
        - damping: The damping parameter.
    """

    def __init__(self, comm=None, ostream=None):
        """
        Initializes complex linear response solver to default setup.
        """

        if comm is None:
            comm = MPI.COMM_WORLD

        if ostream is None:
            if comm.Get_rank() == mpi_master():
                ostream = OutputStream(sys.stdout)
            else:
                ostream = OutputStream(None)

        super().__init__(comm, ostream)

        self.max_iter = 10

        self.a_operator = 'electric dipole'
        self.a_components = 'xyz'
        self.b_operator = 'electric dipole'
        self.b_components = 'xyz'

        self.cpp_flag = None

        self.frequencies = (0.1,)
        #self.damping = 1000.0 / hartree_in_wavenumber()
        self.damping = 0.0045563
        #self.damping = 0

        # Bools for tdasolver
        self.core_excitation = False


        self._input_keywords['response'].update({
            'a_operator': ('str_lower', 'A operator'),
            'a_components': ('str_lower', 'Cartesian components of A operator'),
            'b_operator': ('str_lower', 'B operator'),
            'b_components': ('str_lower', 'Cartesian components of B operator'),
            'frequencies': ('seq_range', 'frequencies'),
            'damping': ('float', 'damping parameter'),
        })

    # def update_settings(self, rsp_dict, method_dict=None):
    #     """
    #     Updates response and method settings in complex liner response solver.

    #     :param rsp_dict:
    #         The dictionary of response input.
    #     :param method_dict:
    #         The dictionary of method settings.
    #     """

    #     if method_dict is None:
    #         method_dict = {}

    #     super().update_settings(rsp_dict, method_dict)

    def set_cpp_flag(self, flag):
        """
        Sets CPP flag (absorption or ecd).

        :param flag:
            The flag (absorption or ecd).
        """

        assert_msg_critical(flag.lower() in ['absorption', 'ecd'],
                            'ComplexResponse: invalide CPP flag')

        self.cpp_flag = flag.lower()

        if self.cpp_flag == 'absorption':
            self.a_operator = 'electric dipole'
            self.a_components = 'xyz'
            self.b_operator = 'electric dipole'
            self.b_components = 'xyz'

        elif self.cpp_flag == 'ecd':
            self.a_operator = 'magnetic dipole'
            self.a_components = 'xyz'
            self.b_operator = 'linear momentum'
            self.b_components = 'xyz'

    def _get_precond(self, orb_ene, nocc, norb, w, d):
        """
        Constructs the preconditioners.

        :param orb_ene:
            The orbital energies.
        :param nocc:
            The number of doubly occupied orbitals.
        :param norb:
            The number of orbitals.
        :param w:
            The frequency.
        :param d:
            The damping parameter.

        :return:
            The distributed preconditioners.
        """

        # spawning needed components

        ediag, sdiag = self.construct_ediag_sdiag_half(orb_ene, nocc, norb)

        ediag_sq = ediag**2
        sdiag_sq = sdiag**2
        w_sq = w**2
        d_sq = d**2
        # constructing matrix block diagonals

        a_diag = ediag - w*sdiag
        b_diag = d * sdiag
        p_diag = 1 / (ediag_sq + w_sq*sdiag_sq - 2*ediag*sdiag + d_sq*sdiag)

        pa_diag = p_diag * a_diag
        pb_diag = p_diag * b_diag

        p_mat = np.hstack((
            pa_diag.reshape(-1, 1),
            pb_diag.reshape(-1, 1),
        ))

        return DistributedArray(p_mat, self.comm)
    
    def _get_precond_identity(self, nocc, norb, w, d):
        """
        Constructs the preconditioners.

        :param orb_ene:
            The orbital energies.
        :param nocc:
            The number of doubly occupied orbitals.
        :param norb:
            The number of orbitals.
        :param w:
            The frequency.
        :param d:
            The damping parameter.

        :return:
            The distributed preconditioners.
        """
        nvir = norb - nocc
        # spawning needed components
       
        ediag = np.ones(nvir*nocc)
        sdiag = np.ones(nvir*nocc)
        ediag_sq = ediag**2
        sdiag_sq = sdiag**2
        w_sq = w**2
        d_sq = d**2
        # constructing matrix block diagonals
        
        a_diag = ediag - w*sdiag
        b_diag = d * sdiag
        p_diag = 1 / (ediag_sq + w_sq*sdiag_sq - 2*ediag*sdiag + d_sq*sdiag)

        pa_diag = p_diag * a_diag
        pb_diag = p_diag * b_diag

        p_mat = np.hstack((
            pa_diag.reshape(-1, 1),
            pb_diag.reshape(-1, 1),
        ))

        return DistributedArray(p_mat, self.comm)    

    def _preconditioning(self, precond, v_in):
        """
        Applies preconditioner to a tuple of distributed trial vectors.

        :param precond:
            The preconditioner.
        :param v_in:
            The input trial vectors.

        :return:
            A tuple of distributed trial vectors after preconditioning.
        """

        pa = precond.data[:, 0]
        pb = precond.data[:, 1]

        v_in_r = v_in.data[:, 0]
        v_in_i = v_in.data[:, 1]

        v_out_r = pa * v_in_r + pb * v_in_i
        v_out_i = pb * v_in_r - pa * v_in_i

        v_mat = np.hstack((
            v_out_r.reshape(-1, 1),
            v_out_i.reshape(-1, 1),
        ))

        return DistributedArray(v_mat, self.comm, distribute=False)
    

    def _precond_trials(self, vectors, precond):
        """
        Applies preconditioner to distributed trial vectors.

        :param vectors:
            The set of vectors.
        :param precond:
            The preconditioner.

        :return:
            The preconditioned gerade and ungerade trial vectors.
        """

        trials = []

        for (op, w), vec in vectors.items():
            v = self._preconditioning(precond[w], vec)
            norms_2 = v.squared_norm(axis=0)
            vn = np.sqrt(np.sum(norms_2))

            if vn > self.norm_thresh:
                norms = np.sqrt(norms_2)
                # real
                if norms[0] > self.norm_thresh:
                    trials.append(v.data[:, 0])
                # imaginary
                if norms[1] > self.norm_thresh:
                    trials.append(v.data[:, 1])

            print("key", op, "norm after precond trial R", np.linalg.norm(v.data[:, 0]), "trial I", np.linalg.norm(v.data[:, 1]))

        new = np.array(trials).T
        
        dist_new = DistributedArray(new, self.comm, distribute=False)

        return dist_new

    def compute(self, molecule, basis, scf_tensors, op_x_drv, op_y_drv, parameters, v_grad=None):
        """
        Solves for the response vector iteratively while checking the residuals
        for convergence.

        :param molecule:
            The molecule.
        :param basis:
            The AO basis.
        :param scf_tensors:
            The dictionary of tensors from converged SCF wavefunction.
        :param v_grad:
            The gradients on the right-hand side. If not provided, v_grad will
            be computed for the B operator.

        :return:
            A dictionary containing response functions, solutions and a
            dictionary containing solutions and kappa values when called from
            a non-linear response module.
        """

        if self.norm_thresh is None:
            self.norm_thresh = self.conv_thresh * 1.0e-6
        if self.lindep_thresh is None:
            self.lindep_thresh = self.conv_thresh * 1.0e-2

        self._dist_b = None
        self._dist_e2b = None

        self.nonlinear = False
        self._dist_fock_ger = None
        self._dist_fock_ung = None
        

        # check molecule
        molecule_sanity_check(molecule)

        # check SCF results
        scf_results_sanity_check(self, scf_tensors)

        # check print level (verbosity of output)
        if self.print_level < 2:
            self.print_level = 1
        if self.print_level > 2:
            self.print_level = 3

        if self.rank == mpi_master():
            self._print_header('Complex Response Solver',
                               n_freqs=len(self.frequencies))

        self.start_time = tm.time()

        # sanity check
        nalpha = molecule.number_of_alpha_electrons()
        nbeta = molecule.number_of_beta_electrons()
        assert_msg_critical(
            nalpha == nbeta,
            'ComplexResponse: not implemented for unrestricted case')

        if self.rank == mpi_master():
            orb_ene = scf_tensors['E_alpha']
        else:
            orb_ene = None
        orb_ene = self.comm.bcast(orb_ene, root=mpi_master())
        norb = orb_ene.shape[0]
        nocc = molecule.number_of_alpha_electrons()

        # ERI information
        eri_dict = self._init_eri(molecule, basis)

        # DFT information
        dft_dict = self._init_dft(molecule, scf_tensors)

        # PE information
        pe_dict = self._init_pe(molecule, basis)

        # right-hand side (gradient)
        if self.rank == mpi_master():
            self.nonlinear = (v_grad is not None)
        self.nonlinear = self.comm.bcast(self.nonlinear, root=mpi_master())

        if not self.nonlinear:
            # b_grad = self.get_complex_prop_grad(self.b_operator,
            #                                     self.b_components, molecule,
            #                                     basis, scf_tensors)
            b_grad  = self.get_complex_prop_grad_tda(self.b_operator, self.b_components, molecule, basis,
                              scf_tensors)
            if self.rank == mpi_master():
                v_grad = {
                    (op, w): v for op, v in zip(self.b_components, b_grad)
                    for w in self.frequencies
                }

        # operators, frequencies and preconditioners
        if self.rank == mpi_master():
            op_freq_keys = list(v_grad.keys())
        else:
            op_freq_keys = None
        op_freq_keys = self.comm.bcast(op_freq_keys, root=mpi_master())

        d = self.damping

        self.frequencies = []
        for (op, w) in op_freq_keys:
            if w not in self.frequencies:
                self.frequencies.append(w)

        precond = {
            w: self._get_precond(orb_ene, nocc, norb, w, d)
            #w: self._get_precond_identity(nocc, norb, w, d)
            for w in self.frequencies
        }

        # distribute the gradient and right-hand side:
        # dist_grad will be used for calculating the subspace matrix
        # equation and residuals, dist_rhs for the initial guess

        dist_grad = {}
        dist_rhs = {}
        for key in op_freq_keys:
            if self.rank == mpi_master():
                #gradger, gradung = self._decomp_grad(v_grad[key])
                grad = v_grad[key]
                grad_mat = np.hstack((
                    grad.real.reshape(-1, 1),
                    grad.imag.reshape(-1, 1),
                ))
                rhs_mat = np.hstack((
                    -grad.real.reshape(-1, 1),
                    grad.imag.reshape(-1, 1),
                )) # RAS these signs are reversed in vlx solver
                print("key",key,"norm rhs R", np.linalg.norm(grad.real), "norm rhs I", np.linalg.norm(grad.imag))
            else:
                grad_mat = None
                rhs_mat = None
   
            dist_grad[key] = DistributedArray(grad_mat, self.comm)
            dist_rhs[key] = DistributedArray(rhs_mat, self.comm)

        if self.nonlinear:
            rsp_vector_labels = [
                'CLR_bger_half_size', 'CLR_bung_half_size',
                'CLR_e2bger_half_size', 'CLR_e2bung_half_size', 'CLR_Fock_ger',
                'CLR_Fock_ung'
            ]
        else:
            rsp_vector_labels = [
                'CLR_bger_half_size', 'CLR_bung_half_size',
                'CLR_e2bger_half_size', 'CLR_e2bung_half_size'
            ]

        # check validity of checkpoint file
        if self.restart:
            if self.rank == mpi_master():
                self.restart = check_rsp_hdf5(self.checkpoint_file,
                                              rsp_vector_labels, molecule,
                                              basis, dft_dict, pe_dict)
            self.restart = self.comm.bcast(self.restart, root=mpi_master())

        # read initial guess from restart file
        if self.restart:
            self._read_checkpoint(rsp_vector_labels)

        # generate initial guess from scratch
        else:
            
            b = self.setup_trials(dist_rhs, precond)

            tdens = self._get_trans_densities(b.data, scf_tensors,
                                                  molecule)     
            fock = self._comp_lr_fock(tdens, molecule, basis, eri_dict,
                                          dft_dict, pe_dict)
            
            if self.rank == mpi_master():

                # if i >= n_restart_iterations:
                sig_mat = self._get_sigmas(fock, scf_tensors, molecule,
                                               b.data)

            self._append_trial_sigma_vectors(b, sig_mat)
   
        
        solutions = {}
        residuals = {}
        relative_residual_norm = {}

        iter_per_trial_in_hours = None

        # start iterations
        for iteration in range(self.max_iter):

            iter_start_time = tm.time()

            xvs = []
            self._cur_iter = iteration

            n = self._dist_b.shape(1)

            e2red = self._dist_b.matmul_AtB(self._dist_e2b) #factor 2 ? 
            
            # s2ug is an identity matrix in the case of CCS
            s2ug = self._dist_b.matmul_AtB(self._dist_b) #factor 2 ? 

            for op, w in op_freq_keys:
                if (iteration == 0 or
                        relative_residual_norm[(op, w)] > self.conv_thresh):

                    grad_r = dist_grad[(op, w)].get_column(0)
                    grad_i = dist_grad[(op, w)].get_column(1)

                    # projections onto gerade and ungerade subspaces:

                    g_real = self._dist_b.matmul_AtB(grad_r) #factor 2 ? 
                    g_imag = self._dist_b.matmul_AtB(grad_i)

                    # creating gradient and matrix for linear equation

                    size = 2 * (n)

                    if self.rank == mpi_master():

                        # gradient

                        g = np.zeros(size)

                        g[:n] = - g_real[:]
                        g[size - n:] = g_imag[:] #RAS signs
                        print("g",g)

                        # matrix

                        mat = np.zeros((size, size))

                        # filling E2gg

                        mat[:n, :n] = e2red[:, :]
                        mat[size - n:, size - n:] = -e2red[:, :]

                        # filling S2ug

                        mat[:n, :n] = -w * s2ug[:, :]

                        mat[size - n:, :n] = d * s2ug[:, :]

                        mat[:n, size - n:] = d * s2ug[:, :]

                        mat[size - n:, size - n:] = w * s2ug[:, :]

                        print("mat",mat)
                        # solving matrix equation

                        c = np.linalg.solve(mat, g)
                        print(f"in iteration {iteration} c is {c}")

                    else:
                        c = None
                    c = self.comm.bcast(c, root=mpi_master())

                    # extracting the 2 components of c...

                    c_real = c[:n]
                    c_imag = c[size - n:]

                    # ...and projecting them onto respective subspace

                    x_real = self._dist_b.matmul_AB_no_gather(c_real)
                    x_imag = self._dist_b.matmul_AB_no_gather(c_imag)
                    
                    # composing E2 matrices projected onto solution subspace

                    e2real = self._dist_e2b.matmul_AB_no_gather(c_real)
                    e2imag = self._dist_e2b.matmul_AB_no_gather(c_imag)

                    # calculating the residual components

                    s2real = x_real.data
                    s2imag = x_imag.data


                    r_real = (e2real.data - w * s2real +
                                 d * s2imag + grad_r.data)
                    r_imag = (-e2imag.data + w * s2imag +
                                 d * s2real - grad_i.data)  # RAS signss ? 

                    r_data = np.hstack((
                        r_real.reshape(-1, 1),
                        r_imag.reshape(-1, 1),
                    ))
                    print(f"residuals: R {np.linalg.norm(r_real)} I {np.linalg.norm(r_imag)}")

                    r = DistributedArray(r_data, self.comm, distribute=False)

                    # calculating relative residual norm
                    # for convergence check

                    x_data = np.hstack((
                        x_real.data.reshape(-1, 1),
                        x_imag.data.reshape(-1, 1),
                    ))

                    x = DistributedArray(x_data, self.comm, distribute=False)

                    x_full = self.get_full_solution_vector(x)
                    
                    print(f" op: {op}, freq {w}, norm R {np.linalg.norm(x_full.real)} norm I {np.linalg.norm(x_full.imag)}")

                    # xv is the DFT response function.. you need to do the CCS one instead RAS
                    if self.rank == mpi_master():
                        xv = np.dot(x_full, v_grad[(op, w)])
                        xvs.append((op, w, xv))

                    r_norms_2 = 2.0 * r.squared_norm(axis=0)
                    x_norms_2 = 2.0 * x.squared_norm(axis=0)

                    rn = np.sqrt(np.sum(r_norms_2))
                    xn = np.sqrt(np.sum(x_norms_2))

                    if xn != 0:
                        relative_residual_norm[(op, w)] = 2.0 * rn / xn
                    else:
                        relative_residual_norm[(op, w)] = 2.0 * rn

                    if relative_residual_norm[(op, w)] < self.conv_thresh:
                        solutions[(op, w)] = x
                    else:
                        residuals[(op, w)] = r
                
            # write to output
            if self.rank == mpi_master():

                self.ostream.print_info(
                    '{:d} trial vectors in reduced space'.format(n))
                self.ostream.print_blank()

                self._print_iteration(relative_residual_norm, xvs)

            # check convergence

            self._check_convergence(relative_residual_norm)

            if self.is_converged:
                break

            # spawning new trial vectors from residuals
            
            new_trials = self.setup_trials(
                residuals, precond, self._dist_b)

            residuals.clear()

            if self.rank == mpi_master():
                n_new_trials = new_trials.shape(1)
            else:
                n_new_trials = None
            n_new_trials = self.comm.bcast(n_new_trials, root=mpi_master())

            if iter_per_trial_in_hours is not None:
                next_iter_in_hours = iter_per_trial_in_hours * n_new_trials
                if self._need_graceful_exit(next_iter_in_hours):
                    self._graceful_exit(molecule, basis, dft_dict, pe_dict,
                                        rsp_vector_labels)

            if self.force_checkpoint:
                self._write_checkpoint(molecule, basis, dft_dict, pe_dict,
                                       rsp_vector_labels)

            # creating new sigma and rho linear transformations

            tdens = self._get_trans_densities(new_trials.data, scf_tensors,
                                                  molecule)     
            fock = self._comp_lr_fock(tdens, molecule, basis, eri_dict,
                                          dft_dict, pe_dict)
            
            if self.rank == mpi_master():

                # if i >= n_restart_iterations:
                sig_mat = self._get_sigmas(fock, scf_tensors, molecule,
                                               b.data)
            
            self._append_trial_sigma_vectors(new_trials, sig_mat)

            iter_in_hours = (tm.time() - iter_start_time) / 3600
            iter_per_trial_in_hours = iter_in_hours / n_new_trials

        self._write_checkpoint(molecule, basis, dft_dict, pe_dict,
                               rsp_vector_labels)

        # converged?
        if self.rank == mpi_master():
            self._print_convergence('Complex response')

        self._dist_b = None
        self._dist_e2b = None

        self._dist_fock_ger = None
        self._dist_fock_ung = None

        # calculate response functions
        if not self.nonlinear:
            a_grad = self.get_complex_prop_grad(self.a_operator,
                                                self.a_components, molecule,
                                                basis, scf_tensors)

            if self.is_converged:
                if self.rank == mpi_master():
                    va = {op: v for op, v in zip(self.a_components, a_grad)}
                    rsp_funcs = {}

                    # create h5 file for response solutions
                    if (self.save_solutions and
                            self.checkpoint_file is not None):
                        final_h5_fname = str(
                            Path(self.checkpoint_file).with_suffix(
                                '.solutions.h5'))
                        create_hdf5(final_h5_fname, molecule, basis,
                                    dft_dict['dft_func_label'],
                                    pe_dict['potfile_text'])

                for bop, w in solutions:
                    x = self.get_full_solution_vector(solutions[(bop, w)])

                    if self.rank == mpi_master():
                        for aop in self.a_components:
                            rsp_funcs[(aop, bop, w)] = -np.dot(va[aop], x)

                        # write to h5 file for response solutions
                        if (self.save_solutions and
                                self.checkpoint_file is not None):
                            solution_keys = [
                                '{:s}_{:s}_{:.8f}'.format(aop, bop, w)
                                for aop in self.a_components
                            ]
                            write_rsp_solution_with_multiple_keys(
                                final_h5_fname, solution_keys, x)

                if self.rank == mpi_master():
                    # print information about h5 file for response solutions
                    if (self.save_solutions and
                            self.checkpoint_file is not None):
                        checkpoint_text = 'Response solution vectors written to file: '
                        checkpoint_text += final_h5_fname
                        self.ostream.print_info(checkpoint_text)
                        self.ostream.print_blank()

                    ret_dict = {
                        'a_operator': self.a_operator,
                        'a_components': self.a_components,
                        'b_operator': self.b_operator,
                        'b_components': self.b_components,
                        'frequencies': list(self.frequencies),
                        'response_functions': rsp_funcs,
                        'solutions': solutions,
                    }

                    self._print_results(ret_dict)

                    return ret_dict
                else:
                    return {'solutions': solutions}

        return None

    @staticmethod
    def get_full_solution_vector(solution):
        """
        Gets a full solution vector from the distributed solution.

        :param solution:
            The distributed solution as a tuple.

        :return:
            The full solution vector.
        """

        x_real = solution.get_full_vector(0)
        x_imag = solution.get_full_vector(1)

        if solution.rank == mpi_master():
            return x_real + 1j * x_imag
        else:
            return None

    def _print_iteration(self, relative_residual_norm, xvs):
        """
        Prints information of the iteration.

        :param relative_residual_norm:
            Relative residual norms.
        :param xvs:
            A list of tuples containing operator component, frequency, and
            property.
        """

        width = 92

        output_header = '*** Iteration:   {} '.format(self._cur_iter + 1)
        output_header += '* Residuals (Max,Min): '
        output_header += '{:.2e} and {:.2e}'.format(
            max(relative_residual_norm.values()),
            min(relative_residual_norm.values()))
        self.ostream.print_header(output_header.ljust(width))
        self.ostream.print_blank()

        if (not self.nonlinear) and (self.print_level > 1):
            output_header = 'Operator:  {} ({})'.format(self.b_operator,
                                                        self.b_components)
            self.ostream.print_header(output_header.ljust(width))
            self.ostream.print_blank()

            for op, freq, xv in xvs:
                ops_label = '<<{};{}>>_{:.4f}'.format(op, op, freq)
                rel_res = relative_residual_norm[(op, freq)]
                output_iter = '{:<15s}: {:15.8f} {:15.8f}j   '.format(
                    ops_label, -xv.real, -xv.imag)
                output_iter += 'Residual Norm: {:.8f}'.format(rel_res)
                self.ostream.print_header(output_iter.ljust(width))
            self.ostream.print_blank()

        self.ostream.flush()
    
    def get_complex_prop_grad_tda(self, operator, components, molecule, basis,
                              scf_tensors):
        """
        Computes complex property gradients for linear response equations.

        :param operator:
            The string for the operator.
        :param components:
            The string for Cartesian components.
        :param molecule:
            The molecule.
        :param basis:
            The AO basis set.
        :param scf_tensors:
            The dictionary of tensors from converged SCF wavefunction.

        :return:
            The complex property gradients.
        """

        # compute 1e integral

        assert_msg_critical(
            operator in [
                'dipole', 'electric dipole', 'electric_dipole',
                'linear_momentum', 'linear momentum', 'angular_momentum',
                'angular momentum', 'magnetic dipole', 'magnetic_dipole'
            ],
            f'LinearSolver.get_complex_prop_grad: unsupported operator {operator}'
        )

        if operator in ['dipole', 'electric dipole', 'electric_dipole']:
            if self.rank == mpi_master():
                dipole_mats = compute_electric_dipole_integrals(
                    molecule, basis, [0.0, 0.0, 0.0])
                integrals = (
                    dipole_mats[0] + 0j,
                    dipole_mats[1] + 0j,
                    dipole_mats[2] + 0j,
                )
            else:
                integrals = tuple()

        # compute right-hand side

        if self.rank == mpi_master():
            indices = {'x': 0, 'y': 1, 'z': 2}
            integral_comps = [integrals[indices[p]] for p in components]

            nocc = molecule.number_of_alpha_electrons()

            if self.core_excitation:
                mo_occ = scf_tensors['C_alpha'][:, :self.
                                                num_core_orbitals].copy()
            else:
                mo_occ = scf_tensors['C_alpha'][:, :nocc].copy()
            mo_vir = scf_tensors['C_alpha'][:, nocc:].copy()
            
            nvir = mo_vir.shape[1]

            matrices = [
                    np.linalg.multi_dot([mo_occ.T, P.T, mo_vir])
                for P in integral_comps
            ]

            gradients = tuple(m.reshape(nocc*nvir) for m in matrices)
            return gradients

        else:
            return tuple()
        
    def _get_sigmas(self, fock, tensors, molecule, trial_mat):
        """
        Computes the sigma vectors.

        :param fock:
            The Fock matrix.
        :param tensors:
            The dictionary of tensors from converged SCF wavefunction.
        :param molecule:
            The molecule.
        :param trial_mat:
            The trial vectors as 2D Numpy array.

        :return:
            The sigma vectors as 2D Numpy array.
        """

        nocc = molecule.number_of_alpha_electrons()
        norb = tensors['C_alpha'].shape[1]
        nvir = norb - nocc

        if self.core_excitation:
            mo_occ = tensors['C_alpha'][:, :self.num_core_orbitals].copy()
        else:
            mo_occ = tensors['C_alpha'][:, :nocc].copy()
        mo_vir = tensors['C_alpha'][:, nocc:].copy()
        orb_ene = tensors['E_alpha']

        sigma_vecs = []
        for fockind in range(len(fock)):
            # 2e contribution
            mat = fock[fockind].copy()
            mat = np.matmul(mo_occ.T, np.matmul(mat, mo_vir))
            # 1e contribution
            if self.core_excitation:
                cjb = trial_mat[:, fockind].reshape(self.num_core_orbitals,
                                                    nvir)
                mat += np.matmul(cjb, np.diag(orb_ene[nocc:]).T)
                mat -= np.matmul(np.diag(orb_ene[:self.num_core_orbitals]), cjb)
                sigma_vecs.append(mat.reshape(self.num_core_orbitals * nvir, 1))
            else:
                cjb = trial_mat[:, fockind].reshape(nocc, nvir)
                mat += np.matmul(cjb, np.diag(orb_ene[nocc:]).T)
                mat -= np.matmul(np.diag(orb_ene[:nocc]), cjb)
                sigma_vecs.append(mat.reshape(nocc * nvir, 1))

        sigma_mat = sigma_vecs[0]
        for vec in sigma_vecs[1:]:
            sigma_mat = np.hstack((sigma_mat, vec))

        return sigma_mat    
    
    def _get_trans_densities(self, trial_mat, tensors, molecule):
        """
        Computes the transition densities.

        :param trial_mat:
            The matrix containing the Z vectors as columns.
        :param tensors:
            The dictionary of tensors from converged SCF wavefunction.
        :param molecule:
            The molecule.

        :return:
            The transition density matrix.
        """

        # form transition densities

        if self.rank == mpi_master():
            nocc = molecule.number_of_alpha_electrons()
            norb = tensors['C_alpha'].shape[1]
            nvir = norb - nocc

            if self.core_excitation:
                mo_occ = tensors['C_alpha'][:, :self.num_core_orbitals].copy()
            else:
                mo_occ = tensors['C_alpha'][:, :nocc].copy()
            mo_vir = tensors['C_alpha'][:, nocc:].copy()

            tdens = []
            for k in range(trial_mat.shape[1]):
                if self.core_excitation:
                    mat = trial_mat[:, k].reshape(self.num_core_orbitals, nvir)
                else:
                    mat = trial_mat[:, k].reshape(nocc, nvir)
                mat = np.matmul(mo_occ, np.matmul(mat, mo_vir.T))
                tdens.append(mat)
        else:
            tdens = None

        return tdens
    
    def _append_trial_sigma_vectors(self, b, e2b):
        """
        Appends distributed trial vectors and sigma vectors.

        :param bger:
            The distributed gerade trial vectors.
        :param bung:
            The distributed ungerade trial vectors.
        """

        if self._dist_b is None:
            self._dist_b = DistributedArray(b.data,
                                               self.comm,
                                               distribute=False)
        else:
            self._dist_b.append(b, axis=1)
        
        if self._dist_e2b is None:
            self._dist_e2b = DistributedArray(e2b,
                                                 self.comm,
                                                 distribute=False)
        else:
            self._dist_e2b.append(e2b, axis=1)
    
    def setup_trials(self,
                      vectors,
                      precond,
                      dist_b=None,
                      renormalize=True):
        """
        Computes orthonormalized trial vectors.

        :param vectors:
            The set of vectors.
        :param precond:
            The preconditioner.
        :param dist_bger:
            The distributed gerade subspace.
        :param dist_bung:
            The distributed ungerade subspace.
        :param renormalize:
            The flag for normalization.

        :return:
            The orthonormalized gerade and ungerade trial vectors.
        """

        dist_new_b = self._precond_trials(vectors, precond)

        if dist_new_b.data.size == 0:
            dist_new_b.data = np.zeros((dist_new_b.shape(0), 0))

        if dist_b is not None:
            # t = t - (b (b.T t))
            bT_new = dist_b.matmul_AtB_allreduce(dist_new_b) # factor 2
            dist_new_proj = dist_b.matmul_AB_no_gather(bT_new)
            dist_new_b.data -= dist_new_proj.data

        if renormalize:
            if dist_new_b.data.ndim > 0 and dist_new_b.shape(0) > 0:
                # dist_new_b = self.remove_linear_dependence(
                #     dist_new_b, self.lindep_thresh)
                dist_new_b = self.orthogonalize_gram_schmidt(
                    dist_new_b)
                dist_new_b = self.normalize(dist_new_b)
                dist_new_b = DistributedArray(dist_new_b, self.comm)

        if self.rank == mpi_master():
            assert_msg_critical(
                dist_new_b.data.size > 0,
                'LinearSolver: trial vectors are empty')

        # if dist_b is not None:
        #     print("old new",np.matmul(dist_b.data.T, dist_new_b.data))
        #     print("new new",np.matmul(dist_new_b.data.T,dist_new_b.data))
            
        return dist_new_b

    def get_spectrum(self, rsp_results, x_unit):
        """
        Gets spectrum.

        :param rsp_results:
            The dictionary containing response results.
        :param x_unit:
            The unit of x-axis.

        :return:
            A dictionary containing the spectrum.
        """

        if self.cpp_flag == 'absorption':
            return self._get_absorption_spectrum(rsp_results, x_unit)

        elif self.cpp_flag == 'ecd':
            return self._get_ecd_spectrum(rsp_results, x_unit)

        return None

    def _get_absorption_spectrum(self, rsp_results, x_unit):
        """
        Gets absorption spectrum.

        :param rsp_results:
            The dictionary containing response results.
        :param x_unit:
            The unit of x-axis.

        :return:
            A dictionary containing the absorption spectrum.
        """

        assert_msg_critical(
            x_unit.lower() in ['au', 'ev', 'nm'],
            'ComplexResponse.get_spectrum: x_unit should be au, ev or nm')

        au2ev = hartree_in_ev()
        auxnm = 1.0 / hartree_in_inverse_nm()

        spectrum = {'x_data': [], 'y_data': []}

        if x_unit.lower() == 'au':
            spectrum['x_label'] = 'Photon energy [a.u.]'
        elif x_unit.lower() == 'ev':
            spectrum['x_label'] = 'Photon energy [eV]'
        elif x_unit.lower() == 'nm':
            spectrum['x_label'] = 'Wavelength [nm]'

        spectrum['y_label'] = 'Absorption cross-section [a.u.]'

        freqs = rsp_results['frequencies']
        rsp_funcs = rsp_results['response_functions']

        for w in freqs:
            if w == 0.0:
                continue

            if x_unit.lower() == 'au':
                spectrum['x_data'].append(w)
            elif x_unit.lower() == 'ev':
                spectrum['x_data'].append(au2ev * w)
            elif x_unit.lower() == 'nm':
                spectrum['x_data'].append(auxnm / w)

            axx = -rsp_funcs[('x', 'x', w)].imag
            ayy = -rsp_funcs[('y', 'y', w)].imag
            azz = -rsp_funcs[('z', 'z', w)].imag

            alpha_bar = (axx + ayy + azz) / 3.0
            sigma = 4.0 * math.pi * w * alpha_bar * fine_structure_constant()

            spectrum['y_data'].append(sigma)

        return spectrum


    def _print_results(self, rsp_results, ostream=None):
        """
        Prints response results to output stream.

        :param rsp_results:
            The dictionary containing response results.
        :param ostream:
            The output stream.
        """

        self._print_response_functions(rsp_results, ostream)

        if self.cpp_flag == 'absorption':
            self._print_absorption_results(rsp_results, ostream)

        elif self.cpp_flag == 'ecd':
            self._print_ecd_results(rsp_results, ostream)

    def _print_response_functions(self, rsp_results, ostream=None):
        """
        Prints response functions to output stream.

        :param rsp_results:
            The dictionary containing response results.
        :param ostream:
            The output stream.
        """

        if ostream is None:
            ostream = self.ostream

        width = 92

        freqs = rsp_results['frequencies']
        rsp_funcs = rsp_results['response_functions']

        title = 'Response Functions at Given Frequencies'
        ostream.print_header(title.ljust(width))
        ostream.print_header(('=' * len(title)).ljust(width))
        ostream.print_blank()

        operator_to_name = {
            'dipole': 'Dipole',
            'electric dipole': 'Dipole',
            'electric_dipole': 'Dipole',
            'linear_momentum': 'LinMom',
            'linear momentum': 'LinMom',
            'angular_momentum': 'AngMom',
            'angular momentum': 'AngMom',
            'magnetic dipole': 'MagDip',
            'magnetic_dipole': 'MagDip',
        }
        a_name = operator_to_name[self.a_operator]
        b_name = operator_to_name[self.b_operator]

        for w in freqs:
            title = '{:<7s} {:<7s} {:>10s} {:>15s} {:>16s}'.format(
                a_name, b_name, 'Frequency', 'Real', 'Imaginary')
            ostream.print_header(title.ljust(width))
            ostream.print_header(('-' * len(title)).ljust(width))

            for a in self.a_components:
                for b in self.b_components:
                    rsp_func_val = rsp_funcs[(a, b, w)]
                    ops_label = '<<{:>3s}  ;  {:<3s}>> {:10.4f}'.format(
                        a.lower(), b.lower(), w)
                    output = '{:<15s} {:15.8f} {:15.8f}j'.format(
                        ops_label, rsp_func_val.real, rsp_func_val.imag)
                    ostream.print_header(output.ljust(width))
            ostream.print_blank()
        ostream.flush()

    def _print_absorption_results(self, rsp_results, ostream=None):
        """
        Prints absorption results to output stream.

        :param rsp_results:
            The dictionary containing response results.
        :param ostream:
            The output stream.
        """

        if ostream is None:
            ostream = self.ostream

        width = 92

        spectrum = self.get_spectrum(rsp_results, 'au')

        title = 'Linear Absorption Cross-Section'
        ostream.print_header(title.ljust(width))
        ostream.print_header(('=' * len(title)).ljust(width))
        ostream.print_blank()

        freqs = rsp_results['frequencies']

        if len(freqs) == 1 and freqs[0] == 0.0:
            text = '*** No linear absorption spectrum at zero frequency.'
            ostream.print_header(text.ljust(width))
            ostream.print_blank()
            return

        title = 'Reference: '
        title += 'J. Kauczor and P. Norman, '
        title += 'J. Chem. Theory Comput. 2014, 10, 2449-2455.'
        ostream.print_header(title.ljust(width))
        ostream.print_blank()

        assert_msg_critical(
            '[a.u.]' in spectrum['x_label'],
            'ComplexResponse._print_absorption_results: In valid unit in x_label'
        )
        assert_msg_critical(
            '[a.u.]' in spectrum['y_label'],
            'ComplexResponse._print_absorption_results: In valid unit in y_label'
        )

        title = '{:<20s}{:<20s}{:>15s}'.format('Frequency[a.u.]',
                                               'Frequency[eV]',
                                               'sigma(w)[a.u.]')
        ostream.print_header(title.ljust(width))
        ostream.print_header(('-' * len(title)).ljust(width))

        for w, sigma in zip(spectrum['x_data'], spectrum['y_data']):
            output = '{:<20.4f}{:<20.5f}{:>13.8f}'.format(
                w, w * hartree_in_ev(), sigma)
            ostream.print_header(output.ljust(width))

        ostream.print_blank()

    

In [4]:
# Notebook calling the solver

op_x_drv = ElectricDipoleMomentDriver()
op_y_drv = ElectricDipoleMomentDriver()

parameters = {}

rsp_driver = ComplexResponse()

def compute(molecule, basis, scf_tensors):
        """
        Computes response property/spectroscopy.

        :param molecule:
            The molecule.
        :param basis:
            The AO basis set.
        :param scf_tensors:
            The dictionary of tensors from converged SCF wavefunction.
        """

        rsp_property = rsp_driver.compute(molecule, basis,
                                                      scf_tensors, op_x_drv, op_y_drv, parameters=parameters)
    
compute(molecule, basis,scf_tensors)


                                                                                                                          
                                              Complex Response Solver Setup                                               
                                                                                                                          
                               Number of Frequencies           : 1                                                        
                               Max. Number of Iterations       : 10                                                       
                               Convergence Threshold           : 1.0e-04                                                  
                               ERI Screening Threshold         : 1.0e-12                                                  
                                                                                                                          


key ('x', 0.1) norm rhs R 0.5196824226709874 norm rhs I 0.0
key ('y', 0.1) norm rhs R 1.9508140322779306 norm rhs I 0.0
key ('z', 0.1) norm rhs R 1.5620639354228014 norm rhs I 0.0
key x norm after precond trial R 3.965102057227318 trial I 0.010012654397144508
key y norm after precond trial R 1.4108856327206967 trial I 0.012728282983253883
key z norm after precond trial R 1.4995673345578708 trial I 0.010373099323194267


TypeError: 'method' object is not subscriptable